<a href="https://colab.research.google.com/github/Emily-KD/PopularityPrediction/blob/main/Data/rarebeauty/Scrape/rarebeauty_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing Instaloader (Take this out if you are running in local machine)
!pip install instaloader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 1.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#library
from google.colab import files
import instaloader
from datetime import datetime
import json
import os
import time

# Configuration (Insert own Username and SessionID)
USERNAME = 'emi_luvsu'
TARGET_PROFILE = 'rarebeauty'
START_DATE = datetime(2024, 1, 1)
END_DATE = datetime(2025, 12, 31)
SESSIONID = '3105556958%3As5Si1dkZRTucDn%3A4%3AAYd41vOIYKjitHkMbMG_2O6RAtd1C1IEw3ZerNEN4w'
DOWNLOAD_FOLDER = f"{TARGET_PROFILE}_images_2024_2025"

# INIT
L = instaloader.Instaloader(
    download_pictures=False,
    download_video_thumbnails=False,
    download_videos=False,
    save_metadata=False,
    compress_json=False
)

# Load session via sessionid cookie
L.context._session.cookies.set('sessionid', SESSIONID)
profile = instaloader.Profile.from_username(L.context, TARGET_PROFILE)

# Create download folder
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

# Scrape Loop
data = []
consecutive_old_posts = 0
print("🔁 Scraping started...")
for idx, post in enumerate(profile.get_posts()):
    post_time = post.date_utc

    if post_time < START_DATE:
        consecutive_old_posts += 1
        print(f"Post {idx+1} is too old ({post_time.date()}), skipping... ({consecutive_old_posts}/3 old posts)")
        if consecutive_old_posts >= 3:
          print("Found 3 older posts in a row — stopping.")
          break
        continue
    else:
        consecutive_old_posts = 0

    # SKIP REELS (standalone video posts)
    if post.is_video and post.typename != 'GraphSidecar':
        print(f"Skipping reel at post {idx+1}")
        continue

    if START_DATE <= post_time <= END_DATE:
        # Get image URLs (including all carousel items)
        if post.typename == 'GraphSidecar':
            image_urls = [node.display_url for node in post.get_sidecar_nodes()]
        else:
            image_urls = [post.url]

        # Store post metadata
        data.append({
            "caption": post.caption or "",
            "timestamp": post_time.isoformat(),
            "likes": post.likes,
            "comments": post.comments,
            "image_urls": image_urls
        })

        # Download each image
        for i, url in enumerate(image_urls):
            try:
                filename = f"{DOWNLOAD_FOLDER}/post{idx+1}_img{i+1}.jpg"
                os.system(f"wget -q \"{url}\" -O \"{filename}\"")
                print(f"Downloaded: {filename}")
            except Exception as e:
                print(f"Error downloading image {i+1} of post {idx+1}: {e}")

        time.sleep(1.5)

# --- SAVE TO JSON ---
json_file = f"/content/drive/Shareddrives/DS340W Project/Dataset NEW*/{TARGET_PROFILE}/{TARGET_PROFILE}_posts_2024_2025.json"
with open(json_file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2)
print(f"JSON saved: {json_file}")

# --- ZIP IMAGES ---
zip_file = f"{DOWNLOAD_FOLDER}.zip"
!zip -r -q {zip_file} {DOWNLOAD_FOLDER}
print(f"Zipped all images to: {zip_file}")

# --- DOWNLOAD FILES TO LOCAL MACHINE ---
files.download(json_file)
files.download(zip_file)


🔁 Scraping started...
Post 1 is too old (2023-10-05), skipping... (1/3 old posts)
Post 2 is too old (2023-04-12), skipping... (2/3 old posts)
Downloaded: rarebeauty_images_2024_2025/post3_img1.jpg
Downloaded: rarebeauty_images_2024_2025/post4_img1.jpg
Downloaded: rarebeauty_images_2024_2025/post4_img2.jpg
Downloaded: rarebeauty_images_2024_2025/post4_img3.jpg
Downloaded: rarebeauty_images_2024_2025/post4_img4.jpg
Downloaded: rarebeauty_images_2024_2025/post4_img5.jpg
Downloaded: rarebeauty_images_2024_2025/post4_img6.jpg
Downloaded: rarebeauty_images_2024_2025/post4_img7.jpg
Downloaded: rarebeauty_images_2024_2025/post4_img8.jpg
Downloaded: rarebeauty_images_2024_2025/post4_img9.jpg
Downloaded: rarebeauty_images_2024_2025/post4_img10.jpg
Downloaded: rarebeauty_images_2024_2025/post4_img11.jpg
Skipping reel at post 5
Skipping reel at post 6
Skipping reel at post 7
Downloaded: rarebeauty_images_2024_2025/post8_img1.jpg
Downloaded: rarebeauty_images_2024_2025/post8_img2.jpg
Downloaded: ra

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>